In [1]:
# to be ignored when copying in the main file
import pandas as pd
import requests
import json
import time

# to be added

import numpy as np
import datetime
from datetime import *
from dateutil.relativedelta import *
import calendar


In [2]:
# importing Wiki csv

wikidata = pd.read_csv('books_per_view.csv')
Wiki =pd.DataFrame(wikidata)
# change the titles to upper case
Wiki['Page'] = Wiki['Page'].str.upper()


In [3]:
#importing NYTimes
NYTimes = pd.read_csv('results_fourcategories.csv')
NYT =pd.DataFrame(NYTimes[['rank', 'title','author','date','month_on_list','list']])
pd.set_option('display.max_rows', 100)
#NYT2 = NYT[['rank', 'title','author','date','month_on_list']]

In [4]:
NYT

,rank,title,author,date,month_on_list,list
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction
2,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction
3,4,WATER FOR ELEPHANTS,Sara Gruen,2011-03-01,5.0,combined-print-and-e-book-fiction
4,5,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,2011-03-01,7.0,combined-print-and-e-book-fiction
...,...,...,...,...,...,...
9400,11,MORE MYSELF,Alicia Keys with Michelle Burford,2020-05-01,1.0,hardcover-nonfiction
9401,12,FORTITUDE,Dan Crenshaw,2020-05-01,2.0,hardcover-nonfiction
9402,13,THE LAST BOOK ON THE LEFT,"Ben Kissel, Marcus Parks and Henry Zebrowski",2020-05-01,1.0,hardcover-nonfiction
9403,14,HELL AND OTHER DESTINATIONS,Madeleine Albright with Bill Woodward,2020-05-01,1.0,hardcover-nonfiction


In [5]:
#Find the number of books in NYT
len(NYT.title.unique())

3457

In [6]:
# Filtering for  just one category to reduce the number of duplicates
NYT = NYT.loc[NYT['list'] == 'combined-print-and-e-book-fiction']
NYT

,rank,title,author,date,month_on_list,list
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction
2,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction
3,4,WATER FOR ELEPHANTS,Sara Gruen,2011-03-01,5.0,combined-print-and-e-book-fiction
4,5,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,2011-03-01,7.0,combined-print-and-e-book-fiction
...,...,...,...,...,...,...
2030,11,THE LAST TRIAL,Scott Turow,2020-06-01,1.0,combined-print-and-e-book-fiction
2031,12,BEACH READ,Emily Henry,2020-06-01,1.0,combined-print-and-e-book-fiction
2032,13,RODHAM,Curtis Sittenfeld,2020-06-01,2.0,combined-print-and-e-book-fiction
2033,14,ALL ADULTS HERE,Emma Straub,2020-06-01,2.0,combined-print-and-e-book-fiction


In [7]:
#Find the number of books in NYT after filtering
len(NYT.title.unique())

1121

In [8]:
#find duplicates in NYT

DuplicatesNYT = NYT[NYT.duplicated(['title'])]
print("Duplicated Rows are :", DuplicatesNYT[['title','date','month_on_list']], sep='\n')

Duplicated Rows are :
                                      title        date  month_on_list
20                      WATER FOR ELEPHANTS  2011-04-01            3.0
25    THE GIRL WHO KICKED THE HORNET’S NEST  2011-04-01            9.0
27          THE GIRL WITH THE DRAGON TATTOO  2011-04-01            8.0
28            THE GIRL WHO PLAYED WITH FIRE  2011-04-01            7.0
29                        CUTTING FOR STONE  2011-04-01            4.0
...                                     ...         ...            ...
2017                        THE DUTCH HOUSE  2020-05-01            2.0
2021                WHERE THE CRAWDADS SING  2020-06-01           37.0
2022                LITTLE FIRES EVERYWHERE  2020-06-01            9.0
2023                          NORMAL PEOPLE  2020-06-01            1.0
2029                          AMERICAN DIRT  2020-06-01            8.0

[914 rows x 3 columns]


###### Dealing with duplicated entres

In [9]:
#1  Checking the reason for duplication based on one book.

NYT.loc[NYT['title'] == 'IT']

,rank,title,author,date,month_on_list,list
1541,2,IT,Stephen King,2017-10-01,2.0,combined-print-and-e-book-fiction
1558,4,IT,Stephen King,2017-11-01,2.0,combined-print-and-e-book-fiction
1906,7,IT,Stephen King,2019-10-01,1.0,combined-print-and-e-book-fiction


Result: The duplication happens becase of different dates the book was placed on the bestsellers list, the duration on the list, and the different rank each placement had. 

In [10]:
#2. Creating a copy of NYT dataset and assigning a new column to store the last month book was on the list. 
NYT_mod1 = NYT.copy()
NYT_mod1['last_month'] = NYT.apply(lambda _: '', axis=1)
NYT_mod1

,rank,title,author,date,month_on_list,list,last_month
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction,
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction,
2,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction,
3,4,WATER FOR ELEPHANTS,Sara Gruen,2011-03-01,5.0,combined-print-and-e-book-fiction,
4,5,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,2011-03-01,7.0,combined-print-and-e-book-fiction,
...,...,...,...,...,...,...,...
2030,11,THE LAST TRIAL,Scott Turow,2020-06-01,1.0,combined-print-and-e-book-fiction,
2031,12,BEACH READ,Emily Henry,2020-06-01,1.0,combined-print-and-e-book-fiction,
2032,13,RODHAM,Curtis Sittenfeld,2020-06-01,2.0,combined-print-and-e-book-fiction,
2033,14,ALL ADULTS HERE,Emma Straub,2020-06-01,2.0,combined-print-and-e-book-fiction,


In [11]:
#3 Calculating End Date
NYT_mod1['date'] = pd.to_datetime(NYT_mod1['date'])
NYT_mod1['last_month'] = pd.to_datetime(NYT_mod1['last_month'])

f = lambda x: x ['date'] + relativedelta(months = int(x['month_on_list']))
NYT_mod1['last_month'] = NYT_mod1.apply(f, axis=1)

NYT_mod1

,rank,title,author,date,month_on_list,list,last_month
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction,2011-04-01
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction,2011-05-01
2,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction,2011-11-01
3,4,WATER FOR ELEPHANTS,Sara Gruen,2011-03-01,5.0,combined-print-and-e-book-fiction,2011-08-01
4,5,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,2011-03-01,7.0,combined-print-and-e-book-fiction,2011-10-01
...,...,...,...,...,...,...,...
2030,11,THE LAST TRIAL,Scott Turow,2020-06-01,1.0,combined-print-and-e-book-fiction,2020-07-01
2031,12,BEACH READ,Emily Henry,2020-06-01,1.0,combined-print-and-e-book-fiction,2020-07-01
2032,13,RODHAM,Curtis Sittenfeld,2020-06-01,2.0,combined-print-and-e-book-fiction,2020-08-01
2033,14,ALL ADULTS HERE,Emma Straub,2020-06-01,2.0,combined-print-and-e-book-fiction,2020-08-01


In [12]:
#4 Finding the earliest start date and the latest end date
later_end_date = (pd.DataFrame(NYT_mod1.groupby('title')['last_month'].max())).rename(columns = {'last_month':'E_Date'})
earlier_start_date = (pd.DataFrame(NYT_mod1.groupby('title')['date'].min())).rename(columns = {'date':'S_Date'})

In [13]:
#5 Merging the obtained dates with the main NYT dataset

obtained_dates = later_end_date.merge(earlier_start_date, on = 'title')

NYT_mod2 = NYT_mod1.copy().merge(obtained_dates, on = 'title')

len(NYT_mod2)


2035

In [14]:
#6 getting rid of  last_month column

NYT_mod2.head()
NYT_mod2 = NYT_mod2.drop(['last_month'], axis = 1)
NYT_mod2.head()

,rank,title,author,date,month_on_list,list,E_Date,S_Date
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction,2011-04-01,2011-03-01
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction,2011-06-01,2011-03-01
2,11,TICK TOCK,James Patterson and Michael Ledwidge,2011-04-01,2.0,combined-print-and-e-book-fiction,2011-06-01,2011-03-01
3,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction,2012-12-01,2011-03-01
4,8,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-04-01,8.0,combined-print-and-e-book-fiction,2012-12-01,2011-03-01


In [15]:
7# Calculating the average rank of each book to reduce the duplicates and storing it in a table

average_rank = pd.DataFrame(NYT_mod2.groupby('title')['rank'].mean()).rename(columns = {'rank': 'NYT_average_rank'})
len(average_rank)

1121

In [16]:
8# Calculating the average duration of books between the entries for individual books and storing it in a table

average_duration=pd.DataFrame(NYT_mod2.groupby('title')['month_on_list'].mean()).rename(columns = {'month_on_list': 'average_duration'})

len(average_duration)

1121

In [17]:
NYT_mod2.head()

,rank,title,author,date,month_on_list,list,E_Date,S_Date
0,1,ALONE,Lisa Gardner,2011-03-01,1.0,combined-print-and-e-book-fiction,2011-04-01,2011-03-01
1,2,TICK TOCK,James Patterson and Michael Ledwidge,2011-03-01,2.0,combined-print-and-e-book-fiction,2011-06-01,2011-03-01
2,11,TICK TOCK,James Patterson and Michael Ledwidge,2011-04-01,2.0,combined-print-and-e-book-fiction,2011-06-01,2011-03-01
3,3,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-03-01,8.0,combined-print-and-e-book-fiction,2012-12-01,2011-03-01
4,8,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,2011-04-01,8.0,combined-print-and-e-book-fiction,2012-12-01,2011-03-01


In [18]:
#8 Dropping original rank, date and month_on_list from the NYT Table

NYT_mod2 =NYT_mod2.drop(['rank'], axis = 1)
NYT_mod2 =NYT_mod2.drop(['month_on_list'], axis = 1)
NYT_mod2 =NYT_mod2.drop(['date'], axis = 1)
len(NYT_mod2.title.unique())

1121

In [19]:
len(NYT_mod2)

2035

In [20]:
NYT_mod2.loc[NYT_mod2['title'] == 'IT']

,title,author,list,E_Date,S_Date
1582,IT,Stephen King,combined-print-and-e-book-fiction,2019-11-01,2017-10-01
1583,IT,Stephen King,combined-print-and-e-book-fiction,2019-11-01,2017-10-01
1584,IT,Stephen King,combined-print-and-e-book-fiction,2019-11-01,2017-10-01


In [21]:
#9 Dropping duplicates
#NYT_mod2.drop_duplicates()
#NYT_mod2= NYT_mod2.loc[~NYT_mod2.index.duplicated(keep='first')]

NYT_mod3 = NYT_mod2.drop_duplicates(['title'], keep='last')

#NYT_mod2.set_index('title', inplace=True)
#NYT_mod2.drop_duplicates(keep=False,inplace=False) 
#NYT_mod2.reset_index(level=0, inplace=True)
# something not right...there should not be duplicates, there should be only one book with this data?!
len(NYT_mod3)


1121

In [22]:
#10 Checking a sample

NYT_mod3.loc[NYT_mod3['title'] == 'IT']



,title,author,list,E_Date,S_Date
1584,IT,Stephen King,combined-print-and-e-book-fiction,2019-11-01,2017-10-01


In [23]:
len(NYT_mod3)

1121

In [24]:
len(NYT_mod3.title.unique())

1121

In [25]:
#11 Merging the calculated rank, and number of months  with the NYT data
NYT_mod4 = NYT_mod3.copy().merge(average_rank, on = 'title').merge(average_duration, on = 'title')
len(NYT_mod4)

1121

In [26]:
NYT_mod4.head()

,title,author,list,E_Date,S_Date,NYT_average_rank,average_duration
0,ALONE,Lisa Gardner,combined-print-and-e-book-fiction,2011-04-01,2011-03-01,1.000000,1.000000
1,TICK TOCK,James Patterson and Michael Ledwidge,combined-print-and-e-book-fiction,2011-06-01,2011-03-01,6.500000,2.000000
2,THE GIRL WITH THE DRAGON TATTOO,Stieg Larsson,combined-print-and-e-book-fiction,2012-12-01,2011-03-01,9.666667,7.222222
3,WATER FOR ELEPHANTS,Sara Gruen,combined-print-and-e-book-fiction,2012-07-01,2011-03-01,7.666667,3.888889
4,THE GIRL WHO PLAYED WITH FIRE,Stieg Larsson,combined-print-and-e-book-fiction,2012-12-01,2011-03-01,12.375000,6.250000


In [30]:
#12 Merging the NYT and Wiki

Combo = pd.merge(Wiki, NYT_mod4, left_on='Page', right_on='title')
Combo

,Unnamed: 0,Rank,Page,Views in millions,title,...,list,E_Date,S_Date,NYT_average_rank,average_duration
0,1,2,FIFTY SHADES OF GREY,49,FIFTY SHADES OF GREY,...,combined-print-and-e-book-fiction,2017-01-01,2012-04-01,8.520000,18.280000
1,8,9,A GAME OF THRONES,27,A GAME OF THRONES,...,combined-print-and-e-book-fiction,2021-04-01,2011-05-01,12.000000,21.086957
2,13,14,THE HANDMAID'S TALE,22,THE HANDMAID'S TALE,...,combined-print-and-e-book-fiction,2020-06-01,2017-03-01,9.375000,8.000000
3,18,17,THE GIRL WITH THE DRAGON TATTOO,20,THE GIRL WITH THE DRAGON TATTOO,...,combined-print-and-e-book-fiction,2012-12-01,2011-03-01,9.666667,7.222222
4,20,21,INFERNO,19,INFERNO,...,combined-print-and-e-book-fiction,2016-12-01,2013-06-01,6.750000,4.375000
5,21,22,IT,18,IT,...,combined-print-and-e-book-fiction,2019-11-01,2017-10-01,4.333333,1.666667


In [27]:
#7 rearraning 

earlier_start_date = (pd.DataFrame(NYT_mod1.groupby('title')['date'].min())).rename(columns = {'date':'S_Date'})

In [29]:
# find unique values

Dataset.title.unique()

NameError: name 'Dataset' is not defined

In [ ]:
#find duplicates

Duplicates = Dataset[Dataset.duplicated(['title'])]
print("Duplicated Rows are :", Duplicates[['title','date','month_on_list']], sep='\n')

In [ ]:
#DATA CLEANING

In [ ]:
Dataset.columns

In [ ]:
#1. Column renaming
Dataset = Dataset.rename(columns = {'Rank':'Wiki_Rank', 'Views in millions': 'Page_Visits','Page':'Wiki_Title', 'rank':'NYT_Rank', 
                                    'title': 'NYT_Title','author':'Author','date':'Start_Date','month_on_list':'Duration'})
Dataset.columns

In [ ]:
#2. Assigning a new column to store the End date
Dataset['End_Date'] = Dataset.apply(lambda _: '', axis=1)

Dataset.columns

In [ ]:
#3 Calculating End Date
Dataset['Start_Date'] = pd.to_datetime(Dataset['Start_Date'])
Dataset['End_Date'] = pd.to_datetime(Dataset['End_Date'])

f = lambda x: x ['Start_Date'] + relativedelta(months = int(x['Duration']))
Dataset['End_Date'] = Dataset.apply(f, axis=1)

Dataset.columns

In [ ]:
#4 Finding the earliest start date and the latest end date
later_end_date = (pd.DataFrame(Dataset.groupby('Wiki_Title')['End_Date'].max())).rename(columns = {'End_Date':'E_Date'})
earlier_start_date = (pd.DataFrame(Dataset.groupby('Wiki_Title')['Start_Date'].min())).rename(columns = {'Start_Date':'S_Date'})

In [ ]:
#5a Incorporating the new info in the main Dataset
Dataset1 = later_end_date.merge(earlier_start_date, on = 'Wiki_Title')

Dataset2 = Dataset.merge(Dataset1, on = 'Wiki_Title')
Dataset2.head() 

#Final_Dataset1 = Final_Dataset1[['Wiki_Rank','NYT_Rank','Wiki_Title','S_Date', 'E_Date', 'Duration_y','Page_Visits']]

#Final_Dataset2.drop_duplicates()

#Final_Dataset1 = Final_Dataset1.groupby(['Wiki_Title'])



In [ ]:
Dataset2 = Dataset2[['Wiki_Title','Page_Visits','Wiki_Rank','NYT_Title','S_Date','E_Date','Duration']]

In [ ]:
Dataset2.head()

In [ ]:
#Dealing with duplicates
#Approach 1:

#Group by average duration

MeanDuration=Dataset.groupby('Wiki_Title')['Duration'].mean()

#Dataset.head()
#Dataset = (Dataset.groupby('Wiki_Title').agg({'Duration': np.average}))
#Dataset = (Dataset.groupby('Wiki_Title').agg('Duration'= np.average))

#Dataset['Months'] = Dataset.apply(lambda _: '', axis=1)
MeanDuration




In [ ]:
Dataset3 = Dataset2.merge(MeanDuration, on = 'Wiki_Title')
Dataset3.head()


In [ ]:
#Clean 'Data'
Final_Dataset1 = (Dataset3.groupby(['Wiki_Title','Page_Visits','S_Date','E_Date','Duration_y'])['Page_Visits'].mean())
Final_Dataset1

In [ ]:
#Approach 2: Find out the maximum duration amongst the duplicates

In [ ]:
# Creating a new column to store the duration between the earliest S_Date and the latest E_Date
Dataset2['Total_Months'] = Dataset.apply(lambda _: '', axis=1)

Dataset2.columns

In [ ]:
# Recalculating the duration

f = lambda x: x ['E_Date'] - ['S_Date']
Dataset2['Total_Months'] = Dataset2.apply(f, axis=1)

#f = lambda x: x ('S_Date' - 'E_Date').days
#Dataset2['Total_Months'] = Dataset2.apply(f, axis=1)

#f = lambda x: x (['S_Date'] - ['E_Date']).days
#Dataset2['Total_Months'] = Dataset2.apply(f, axis=1)


In [ ]:
type(Dataset2['S_Date'][0])
type(Dataset2['E_Date'][0])

In [ ]:
ADD IT AT THE END
# Reindexing
#Final_Dataset =Final_Dataset.reset_index(drop=True)
# Renaming Columns                                         
#Final_Dataset = Final_Dataset.rename(columns = {'Wiki_Title':'Title','S_Date':'From','E_Date':'To'})                                         
#Final_Dataset
#column_order = ['Wiki_Rank','NYT_Rank','Wiki_Title','NYT_Title','Author','Start_Date','End_Date','Duration','Page_Visits']
#Dataset = Dataset[column_order]
#Dataset["Duration"] = Dataset["Duration"].astype(int)
#del Final_Dataset['NYT_Title']
#del Final_Dataset['End_Date']
#del Final_Dataset['Start_Date']


In [ ]:
# Create a list of Unique books in NYT

NYT_list = NYT.title.unique()

In [ ]:
len(NYT_list)

In [ ]:
#removing duplicated column
Dataset3 = Dataset3.drop(['Duration_x'], axis = 1)

Dataset3.head()


#alternative: Dataset3.drop(Dataset3.filter(regex ='_x$').columns.tolist(),axis = 1, inplace = True)
#Dataset3.head()